This script imputes Supplemental Nutrition Assistance Program (SNAP) recipients dollar benefit amount to match the aggregates with United States Department of Agriculture (USDA) statistics for SNAP. In this current version, we used 2014 CPS data and USDA FY2014 annual reports on SNAP. Please refer to the documentation in the same folder for more details on methodology and assumptions. The output this script is a personal level dataset that contains CPS household level sequence (h_seq), individual participation indicator (snap_participation, 0 - not a recipient, 1 - current recipient on file, 2 - imputed recipient), and benefit amount.

Input: 2014 CPS (cpsmar2014t.csv), number of recipients and their benefits amount by state in 2014 (Administrative.csv)

Output: SNAP_Imputation.csv

Additional Source links: http://www.fns.usda.gov/pd/supplemental-nutrition-assistance-program-snap (zipfile for FY69 through FY16)

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import random
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt

In [2]:
#Variables we use in CPS:
#columns_to_keep = ['hfoodsp','hfdval','h_numper','h_seq','a_age','marsupwt','moop','hhi_yn','chsp_val',
                  # 'gestfips','pedisout','pedisdrs','pedisear', 'pedisrem','pediseye', 'pedisphy', 
                  # 'vet_typ1','vet_yn','pemlr','filestat','wsal_val','semp_val','frse_val',
                  # 'ss_val','rtm_val','oi_val','oi_off','int_yn','uc_yn', 'uc_val','int_val','hfoodmo']

In [3]:
CPS_dataset = pd.read_csv('C:\Users\wangy\OneDrive\Documents\BasicIncomeProject\SNAP.csv')

In [4]:
#recipient or not
CPS_dataset.hfoodsp = np.where(CPS_dataset.hfoodsp == "Yes",1,0)

In [5]:
#Prepare household level data
household_SNAP = CPS_dataset.groupby('h_seq')['hfoodsp'].mean()
household_size = CPS_dataset.groupby('h_seq')['h_numper'].mean()

In [6]:
#Earned income
wage = pd.to_numeric(np.where(CPS_dataset.wsal_val!= 'None or not in universe', CPS_dataset.wsal_val, 0))
self_employed1 = pd.to_numeric(np.where(CPS_dataset.semp_val!= 'None or not in universe', CPS_dataset.semp_val, 0))
self_employed2 = pd.to_numeric(np.where(CPS_dataset.frse_val!= 'None or not in universe', CPS_dataset.frse_val, 0))
p_earned = wage + self_employed1 + self_employed2 #individual earned income
p_earned = 0.8 * p_earned #for net income calculation, there is a 20% deduction in earned income
CPS_dataset['p_earned'] = p_earned

In [7]:
#Unearned income
ss = pd.to_numeric(np.where(CPS_dataset.ss_val!='None or not in universe', CPS_dataset.ss_val, 0))
pension = pd.to_numeric(np.where(CPS_dataset.rtm_val!='None or not in universe', CPS_dataset.rtm_val, 0))
disability = pd.to_numeric(np.where(CPS_dataset.oi_off=='State disability payments', CPS_dataset.oi_val, 0))
unemploy = pd.to_numeric(np.where(CPS_dataset.uc_yn=='Yes', CPS_dataset.uc_val, 0))
interest = pd.to_numeric(np.where(CPS_dataset.int_yn=='Yes', CPS_dataset.int_val, 0))
p_unearned = ss + pension + disability + unemploy + interest #individual unearned income
CPS_dataset['p_unearned'] = p_unearned

In [8]:
#Net Income
CPS_dataset['hh_net_income'] = p_earned + p_unearned
hh_net_income = CPS_dataset.groupby('h_seq')['hh_net_income'].sum()

In [9]:
hh_SNAP = DataFrame(household_SNAP.transpose())

In [10]:
hh_SNAP['hh_net'] = hh_net_income
hh_SNAP['hh_size'] = household_size

In [11]:
hh_SNAP.columns = ['indicator', 'hh_net','hh_size'] #indicator is hfoodsp, a dummy variable

In [12]:
#net income deduction
hh_SNAP.hh_net = np.where(hh_SNAP.hh_size <=3, hh_SNAP.hh_net-155*12, hh_SNAP.hh_net)
hh_SNAP.hh_net = np.where(hh_SNAP.hh_size ==4, hh_SNAP.hh_net-168*12, hh_SNAP.hh_net)
hh_SNAP.hh_net = np.where(hh_SNAP.hh_size ==5, hh_SNAP.hh_net-197*12, hh_SNAP.hh_net)
hh_SNAP.hh_net = np.where(hh_SNAP.hh_size >=6, hh_SNAP.hh_net-226*12, hh_SNAP.hh_net)

In [13]:
#medical deduction
#age over 60
CPS_dataset.a_age = np.where(CPS_dataset.a_age == "80-84 years of age",
                             random.randrange(80, 84),
                             CPS_dataset.a_age)
CPS_dataset.a_age = np.where(CPS_dataset.a_age == "85+ years of age",
                             random.randrange(85, 95),
                             CPS_dataset.a_age)
CPS_dataset.a_age = pd.to_numeric(CPS_dataset.a_age)
#disabled
CPS_dataset['disability'] = np.zeros(len(CPS_dataset))
CPS_dataset.disability = np.where(CPS_dataset.pedisdrs == 'Yes', 1, CPS_dataset.disability)
CPS_dataset.disability = np.where(CPS_dataset.pedisear == 'Yes', 1, CPS_dataset.disability)
CPS_dataset.disability = np.where(CPS_dataset.pediseye == 'Yes', 1, CPS_dataset.disability)
CPS_dataset.disability = np.where(CPS_dataset.pedisout == 'Yes', 1, CPS_dataset.disability)
CPS_dataset.disability = np.where(CPS_dataset.pedisphy == 'Yes', 1, CPS_dataset.disability)
CPS_dataset.disability = np.where(CPS_dataset.pedisrem == 'Yes', 1, CPS_dataset.disability)
#deduction of more than $35 for a month
CPS_dataset.hhi_yn = np.where(CPS_dataset.hhi_yn == 'Yes', 1, 0)
CPS_dataset.moop = np.where(CPS_dataset.moop != 'NIU', CPS_dataset.moop, 0)
CPS_dataset['medical'] = np.zeros(len(CPS_dataset))
CPS_dataset.medical = np.where((CPS_dataset.hhi_yn == 0), CPS_dataset.moop, 0)
CPS_dataset.medical = np.where((CPS_dataset.disability == 1.0)|(CPS_dataset.a_age >= 60), CPS_dataset.moop, 0)
CPS_dataset.medical = pd.to_numeric(CPS_dataset.medical)
hh_medical = CPS_dataset.groupby('h_seq')['medical'].sum()
hh_SNAP['hh_medical'] = hh_medical
hh_SNAP.hh_net = np.where(hh_medical > 35*12, hh_SNAP.hh_net-(hh_medical-35*12), hh_SNAP.hh_net)

In [14]:
#child support
CPS_dataset.chsp_val = np.where(CPS_dataset.chsp_val == "NIU", 0, CPS_dataset.chsp_val)
CPS_dataset.chsp_val = pd.to_numeric(CPS_dataset.chsp_val)
hh_child = CPS_dataset.groupby('h_seq')['chsp_val'].sum()
hh_SNAP['hh_child'] = hh_child
hh_SNAP.hh_net = hh_SNAP.hh_net - hh_SNAP.hh_child

In [15]:
hh_SNAP.hh_net = hh_SNAP.hh_net - (10+290)*12 #dependent care; excess shelter. All based on SNAP official average
hh_SNAP.hh_net = np.where(hh_SNAP.hh_net <0, 0, hh_SNAP.hh_net)

In [16]:
#Keep a reasonable subset
hh_SNAP.hh_net = np.where(hh_SNAP.hh_net > 5490 * 12, -100, hh_SNAP.hh_net)
hh_SNAP = hh_SNAP[hh_SNAP.hh_net>=0]

In [17]:
#Regression
model = sm.ols(formula='indicator ~ hh_net', data=hh_SNAP).fit()
print model.summary()

                            OLS Regression Results                            
Dep. Variable:              indicator   R-squared:                       0.121
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     5187.
Date:                Sat, 19 Nov 2016   Prob (F-statistic):               0.00
Time:                        22:07:20   Log-Likelihood:                -13579.
No. Observations:               37780   AIC:                         2.716e+04
Df Residuals:                   37778   BIC:                         2.718e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.3364      0.003    112.407      0.0

In [18]:
#Keep household with 1 member
probs = model.fittedvalues[hh_SNAP.hh_size==1]

In [19]:
#Prepare household's weights; states; SNAP value
household_marsupwt = CPS_dataset.groupby('h_seq')['marsupwt'].mean()
hh_SNAP['hh_marsupwt'] = household_marsupwt

household_gestfips = CPS_dataset.groupby('h_seq')['gestfips'].mean()
hh_SNAP['hh_gestfips'] = household_gestfips

CPS_dataset.hfdval = np.where(CPS_dataset.hfdval!= 'Not in universe', CPS_dataset.hfdval, 0)
CPS_dataset.hfdval = pd.to_numeric(CPS_dataset.hfdval)
household_hfdval = CPS_dataset.groupby('h_seq')['hfdval'].mean()
hh_SNAP['hh_hfdval'] = household_hfdval


In [20]:
#Prepare household's length of being recipients
CPS_dataset.hfoodmo = np.where(CPS_dataset.hfoodmo == 'Not in universe', 0, CPS_dataset.hfoodmo)
CPS_dataset.hfoodmo = np.where(CPS_dataset.hfoodmo == '12 Months', 12, CPS_dataset.hfoodmo)
CPS_dataset.hfoodmo = np.where(CPS_dataset.hfoodmo == '1 month', 1, CPS_dataset.hfoodmo)
CPS_dataset.hfoodmo = pd.to_numeric(CPS_dataset.hfoodmo)
household_hfoodmo = CPS_dataset.groupby('h_seq')['hfoodmo'].mean()
hh_SNAP['hh_hfoodmo'] = household_hfoodmo

In [21]:
#Import administrative level data
admin = pd.read_csv('C:\Users\wangy\OneDrive\Documents\BasicIncomeProject\Administrative.csv',
                    dtype={ 'Household number': np.float,'Average household benefit': np.float, 'Total': np.float, 'Individual number': np.float})
admin.index = admin.Fips

In [22]:
# CPS total benefits and Administrative total benefits
state_benefit = {}
state_recipients = {}
for state in admin.Fips:
    this_state = (hh_SNAP.hh_gestfips==state)
    CPS_totalb = (hh_SNAP.hh_hfdval * hh_SNAP.hh_marsupwt / hh_SNAP.hh_hfoodmo)[this_state].sum()/1000000 #in million, per month
    admin_totalb =  admin.Total[state] /12 /1000000 # in million, per month
    CPS_totaln = (hh_SNAP.hh_marsupwt[this_state&hh_SNAP.indicator==1]*hh_SNAP.hh_hfoodmo/12).sum()/1000 # in thousand, per month
    admin_totaln =  admin["Household number"][state] /1000 #household in thousand, per month
    CPS_totalnindividual = (hh_SNAP.hh_marsupwt* hh_SNAP.hh_size[this_state&hh_SNAP.indicator==1]*hh_SNAP.hh_hfoodmo/12).sum()/1000 # in thousand, per month
    admin_totalnindividual =  admin["Individual number"][state] /1000 #individual in thousand, per month
    
    temp = [admin.State[state], CPS_totalb, admin_totalb, CPS_totaln, admin_totaln, CPS_totalnindividual, admin_totalnindividual]
    state_benefit[state] = temp
    
pre_augment_benefit = DataFrame(state_benefit).transpose()
pre_augment_benefit.columns = ['State', 'CPS total benefits(monthly)','Admin total benefits(monthly)',
                               'CPS total household recipient(monthly)','Admin total household recipient(monthly)',
                               'CPS total individual recipient(monthly)','Admin total individual recipient(monthly)']

In [23]:
pre_augment_benefit.to_csv('C:\Users\wangy\OneDrive\Documents\BasicIncomeProject\pre-blow-up.csv')

In [24]:
# caculate difference of SNAP stats and CPS aggregates on recipients number
# by state
diff = {'Fips':[],'Difference in Population':[],'Mean Benefit':[],'CPS Population':[],'SNAP Population':[]}
diff['Fips'] = admin.Fips
current = (hh_SNAP.indicator==1)
for FIPS in admin.Fips:
        this_state = (hh_SNAP.hh_gestfips==FIPS)
        current_tots = (hh_SNAP.hh_marsupwt[current&this_state]*hh_SNAP.hh_hfoodmo/12).sum()
        valid_num = (hh_SNAP.hh_marsupwt[current&this_state]*hh_SNAP.hh_hfoodmo/12).sum() + 0.0000001
        current_mean = ((hh_SNAP.hh_hfdval * hh_SNAP.hh_marsupwt / hh_SNAP.hh_hfoodmo)[current&this_state].sum())/valid_num
        diff['CPS Population'].append(current_tots)
        diff['SNAP Population'].append(float(admin["Household number"][admin.Fips == FIPS]))
        diff['Difference in Population'].append(float(admin["Household number"][admin.Fips == FIPS])- current_tots)
        diff['Mean Benefit'].append(current_mean)

In [25]:
d = DataFrame(diff)
d = d[['Fips', 'Mean Benefit', 'Difference in Population', 'CPS Population', 'SNAP Population']]
d.to_csv('recipients.csv', index=False)

In [26]:
hh_SNAP['impute'] = np.zeros(len(hh_SNAP))
hh_SNAP['snap_impute'] = np.zeros(len(hh_SNAP))

non_current = (hh_SNAP.indicator==0)
current = (hh_SNAP.indicator==1)
random.seed()

for FIPS in admin.Fips:
    
        print ('we need to impute', d['Difference in Population'][FIPS], 'for state', FIPS)
        
        if d['Difference in Population'][FIPS] < 0:
            continue
        else:
            this_state = (hh_SNAP.hh_gestfips==FIPS)
            not_imputed = (hh_SNAP.impute==0)
            pool_index = hh_SNAP[this_state&not_imputed&non_current].index
            pool = DataFrame({'weight': hh_SNAP.hh_marsupwt[pool_index], 'prob': probs[pool_index]},
                            index=pool_index)
            pool = pool.sort(columns='prob', ascending=False)
            pool['cumsum_weight'] = pool['weight'].cumsum()
            pool['distance'] = abs(pool.cumsum_weight-d['Difference in Population'][FIPS])
            min_index = pool.sort(columns='distance')[:1].index
            min_weight = int(pool.loc[min_index].cumsum_weight)
            pool['impute'] = np.where(pool.cumsum_weight<=min_weight+10 , 1, 0)
            hh_SNAP.impute[pool.index[pool['impute']==1]] = 1
            hh_SNAP.snap_impute[pool.index[pool['impute']==1]] = admin['Average household benefit'][FIPS]*12

        print ('Method1: regression gives', 
                hh_SNAP.hh_marsupwt[(hh_SNAP.impute==1)&this_state].sum()) 

('we need to impute', 193971.92316468252, 'for state', 1)
('Method1: regression gives', 194801.16999999993)
('we need to impute', 16953.308549603178, 'for state', 2)


C:\Users\wangy\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Users\wangy\Anaconda2\lib\site-packages\ipykernel\__main__.py:23: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Users\wangy\Anaconda2\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\wangy\Anaconda2\lib\site-packages\ipykernel\__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('Method1: regression gives', 16934.76)
('we need to impute', 121834.77696626983, 'for state', 4)
('Method1: regression gives', 122896.3)
('we need to impute', 69548.836071428552, 'for state', 5)
('Method1: regression gives', 68166.65)
('we need to impute', 1054907.1063809535, 'for state', 6)
('Method1: regression gives', 1054078.9699999993)
('we need to impute', 91129.277058531821, 'for state', 8)
('Method1: regression gives', 91346.78)
('we need to impute', 111685.27889285717, 'for state', 9)
('Method1: regression gives', 111345.62000000002)
('we need to impute', 31544.582886904755, 'for state', 10)
('Method1: regression gives', 31680.62)
('we need to impute', 42121.886740079382, 'for state', 11)
('Method1: regression gives', 42171.719999999994)
('we need to impute', 1105843.3499348853, 'for state', 12)
('Method1: regression gives', 1106734.5799999994)
('we need to impute', 478369.01947222208, 'for state', 13)
('Method1: regression gives', 477010.41000000003)
('we need to impute', 52

In [27]:
#Adjustment ratio
results = {}

imputed = (hh_SNAP.impute == 1)
has_val = (hh_SNAP.hh_hfdval != 0)
no_val = (hh_SNAP.hh_hfdval == 0)

for FIPS in admin.Fips:
    this_state = (hh_SNAP.hh_gestfips==FIPS)
    
    current_total = (hh_SNAP.hh_hfdval * hh_SNAP.hh_marsupwt)[this_state].sum() #yearly benefit
    imputed_total = (hh_SNAP.snap_impute * hh_SNAP.hh_marsupwt)[this_state&imputed].sum()
    on_file = current_total + imputed_total

    admin_total = admin.Total[FIPS]
    
    adjust_ratio = admin_total / on_file
    this_state_num = [admin['State'][FIPS], on_file, admin_total, adjust_ratio]
    results[FIPS] = this_state_num
    

    hh_SNAP.snap_impute = np.where(has_val&this_state, hh_SNAP.hh_hfdval * adjust_ratio, hh_SNAP.snap_impute)
    hh_SNAP.snap_impute = np.where(no_val&this_state, hh_SNAP.snap_impute * adjust_ratio, hh_SNAP.snap_impute)

hh_SNAP["snap_participation"] = np.zeros(len(hh_SNAP))
hh_SNAP["snap_participation"] = np.where(hh_SNAP.impute==1, 2, 0)
hh_SNAP["snap_participation"] = np.where(has_val, 1, hh_SNAP.snap_participation)


r = DataFrame(results).transpose()
r.columns=['State', 'Imputed', 'Admin', 'adjust ratio']
r.to_csv('amount.csv', index=False)

In [28]:
hh_SNAP.to_csv('SNAP_Imputation.csv', 
                   columns=['snap_participation', 'snap_impute'])

In [29]:
#Individual check
imputed1 = (hh_SNAP.snap_participation == 1)
imputed2 = (hh_SNAP.snap_participation == 2)

result = {}

for FIPS in admin.Fips:
    this_state = (hh_SNAP.hh_gestfips==FIPS)
    admin_totalindividual =  admin["Individual number"][FIPS]
    imputed_totalindividual = (hh_SNAP.hh_marsupwt* hh_SNAP.hh_size*hh_SNAP.hh_hfoodmo/12)[this_state&imputed1].sum()+(hh_SNAP.hh_marsupwt*hh_SNAP.hh_size)[this_state&imputed2].sum()
    this_state_numindividual = [admin['State'][FIPS], admin_totalindividual, imputed_totalindividual]
    results[FIPS] = this_state_numindividual
    
r = DataFrame(results).transpose()
r.columns=['State', 'admin_totalindividual', 'imputed_totalindividual']
r.to_csv('check_individual.csv', index=False)

In [30]:
#Household size distribution
result = {}
household_weight = CPS_dataset.groupby('h_numper')['marsupwt'].sum()
householdDistribution = DataFrame(household_weight.transpose())
householdDistribution.columns = ['Household number']

householdDistribution['Percentage'] = 100*householdDistribution['Household number']/householdDistribution['Household number'].sum()
householdDistribution['Accumulative number'] = householdDistribution['Household number'].cumsum()
householdDistribution['Accumulative percentage'] = 100*householdDistribution['Accumulative number']/householdDistribution['Household number'].sum()
householdDistribution.to_csv('household_size.csv', index_label='Household size')